In [1]:
#!pip install --no-cache-dir numpy opencv-python opencv-contrib-python mediapipe


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
    --------------------------------------- 0.5/39.5 MB 1.5 MB/s eta 0:00:26
    --------------------------------------- 0.8/39.5 MB 1.3 MB/s eta 0:00:30
   - -------------------------------------- 1.0/39.5 MB 1.3 MB/s eta 0:00:31
   - -------------------------------------- 1.6/39.5 MB 1.4 MB/s eta 0:00:27
   -- ------------------------------------- 2.1/39.5 MB 1.6 MB/s eta 0:00:24
   -- ------------------------------------- 2.1/39.5 MB 1.6 MB/s eta 0:00:24
   -- ------------------------------------- 2.4/39.5 MB 1.4 MB/s eta 0:00:26
   -- ------------------------------------- 2.6/39.5 MB 1.4 MB/s eta 0:00:28
   --- ------------------------------------ 3.1/39.5 MB 1.5 MB/s eta 0:00:25
   --- ------------------------------------ 3.4/39.5 MB 1.5 MB/s eta 0:00:24
   --- -------------

In [5]:
import mediapipe as mp
print(mp.__version__)


0.10.21


In [6]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.8, max_num_hands=2)

# OpenCV video capture 
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Width
cap.set(4, 720)   # Height

#  save the video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (1280, 720))

# Finger tip landmarks
finger_tips = [4, 8, 12, 16, 20]

# Thumb checking 
def is_thumb_open(landmarks, hand_type):
    if hand_type == "Right":  # Right Hand
        return landmarks[4].x < landmarks[3].x
    else:  # Left Hand
        return landmarks[4].x > landmarks[3].x

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Fliping
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    total_fingers = 0  

    if results.multi_hand_landmarks:
        for hand_landmarks, hand_info in zip(results.multi_hand_landmarks, results.multi_handedness):
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get landmark positions
            landmarks = hand_landmarks.landmark
            fingers = []

            # Identify hand (Right or Left)
            hand_type = hand_info.classification[0].label

            # Thumb detection
            fingers.append(1 if is_thumb_open(landmarks, hand_type) else 0)

            # Other four fingers detection
            for tip in finger_tips[1:]:
                if landmarks[tip].y < landmarks[tip - 2].y:  # If tip is above lower joint
                    fingers.append(1)
                else:
                    fingers.append(0)

            hand_finger_count = fingers.count(1)
            total_fingers += hand_finger_count

            # Show finger count for each hand
            hand_label = f"{hand_type} Hand: {hand_finger_count}"
            position = (50, 100 if hand_type == "Right" else 150)
            cv2.putText(frame, hand_label, position, cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    # Display total finger count
    cv2.putText(frame, f'Total Fingers: {total_fingers}', (50, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 4)

    # Save the video frame
    out.write(frame)

    
    cv2.imshow("Hand Tracking & Finger Counting", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()
